In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# auto complete
%config Completer.use_jedi = False

.\heart-dataset.ipynb
.\heart.csv
.\info.txt
.\o2Saturation.csv


In [10]:
# not used?
# o2Saturation_df = pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/o2Saturation.csv')
# o2Saturation_df.head()

heart_df = pd.read_csv('./heart.csv')
heart_df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [11]:
len(heart_df) #303
heart_df.describe()
# 303 records for all cols => no missing values

# only numerical values (categorical features are already converted)

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [12]:
features = ['age', 'sex', 'cp', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh','exng', 'oldpeak', 'slp', 'caa', 'thall']
output = ['output']

X = heart_df[features]
y = heart_df[output]

In [13]:
# libraries
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    X,
    y,
    train_size=0.8,
    test_size=0.2,
    random_state=0
)

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [15]:
log1 = LogisticRegression(penalty='l2', solver='sag')
log2 = LogisticRegression(penalty='none', solver='sag')
log3 = LogisticRegression(penalty='l1', solver='liblinear')
log4 = LogisticRegression(penalty='l2', solver='liblinear')

svc1 = SVC(kernel='linear')
svc2 = SVC(kernel='poly')
svc3 = SVC(kernel='rbf')
svc4 = SVC(kernel='sigmoid')

sgd1 = SGDClassifier(loss='hinge')
sgd2 = SGDClassifier(loss='log')

tree1 = DecisionTreeClassifier(max_leaf_nodes=2)

models = [
    log1,
    log2,
    log3,
    log4,
    svc1,
    svc2,
    svc3,
    svc4,
    sgd1,
    sgd2,
    tree1
]

In [17]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

results = pd.DataFrame(columns=['Model', 'MAE', 'R2'])

for model in models:
    print(model)
    
    model.fit(X_train, y_train.values.ravel())
    
    preds = model.predict(X_valid)

    results.loc[len(results)] = [
        str(model),
        round(mean_absolute_error(y_valid, preds),6),
        round(r2_score(y_valid, preds),6)
    ]

results.sort_values(by='MAE')

LogisticRegression(solver='sag')
LogisticRegression(penalty='none', solver='sag')
LogisticRegression(penalty='l1', solver='liblinear')
LogisticRegression(solver='liblinear')
SVC(kernel='linear')


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


SVC(kernel='poly')
SVC()
SVC(kernel='sigmoid')
SGDClassifier()
SGDClassifier(loss='log')
DecisionTreeClassifier(max_leaf_nodes=2)


,Model,MAE,R2
2,"LogisticRegression(penalty='l1', solver='libli...",0.147541,0.401961
3,LogisticRegression(solver='liblinear'),0.147541,0.401961
4,SVC(kernel='linear'),0.180328,0.269063
10,DecisionTreeClassifier(max_leaf_nodes=2),0.213115,0.136166
0,LogisticRegression(solver='sag'),0.245902,0.003268
1,"LogisticRegression(penalty='none', solver='sag')",0.245902,0.003268
5,SVC(kernel='poly'),0.278689,-0.129630
6,SVC(),0.311475,-0.262527
7,SVC(kernel='sigmoid'),0.442623,-0.794118
9,SGDClassifier(loss='log'),0.442623,-0.794118
